In [1]:
import os
from pymongo import MongoClient

### Connection to MongoDB

In [ ]:
mongodb_uri = os.environ.get('MONGODB_URI')
database_name = os.environ.get('DATABASE_NAME')

client = MongoClient(mongodb_uri)
database = client[database_name]

#### Connecting to transcriptions collection

In [ ]:
transcriptions = database['transcriptions']

#### Drop transcription docs with no audio clip info in audio clips collection

In [ ]:
pipeline = [
    {
        '$lookup': {
            'from': 'modified_audio_clips_xx',     # last updated audio clips collection
            'localField': 'audio_clip', 
            'foreignField': '_id', 
            'as': 'transcript_ac'
        }
    }, {
        '$match': {
            'transcript_ac': {
                '$ne': []
            }
        }
    }, {
        '$project': {
            'transcript_ac': 0
        }
    }, {
        '$out': 'modified_transcriptions'       # new collection
    }
]

transcriptions.aggregate(pipeline)